In [1]:
import pandas as pd
import numpy as np
import os

from ddf_utils.str import to_concept_id

from ddf_utils.factory import oecd

In [2]:
import json

In [5]:
oecd.bulk_download('./source/', 'TABLE5')

In [8]:
source = 'source/TABLE5.json'

In [21]:
data = json.load(open(source))

In [22]:
data.keys()

dict_keys(['header', 'dataSets', 'structure'])

In [23]:
data['dataSets'][0]['series'].keys()

dict_keys(['0:0:0:0', '0:1:0:0', '0:2:0:0', '0:3:0:0', '0:4:0:0', '0:5:0:0', '0:6:0:0', '0:7:0:0', '0:8:0:0', '0:9:0:0', '0:10:0:0', '0:11:0:0', '0:12:0:0', '0:13:0:0', '0:14:0:0', '0:15:0:0', '0:16:0:0', '0:17:0:0', '0:18:0:0', '0:19:0:0', '0:20:0:0', '0:21:0:0', '0:22:0:0', '0:23:0:0', '0:24:0:0', '0:25:0:0', '0:26:0:0', '0:27:0:0', '0:28:0:0', '0:29:0:0', '0:30:0:0', '0:31:0:0', '0:32:0:0', '0:33:0:0', '0:34:0:0', '0:35:0:0', '0:36:0:0', '0:37:0:0', '0:38:0:0', '0:39:0:0', '0:40:0:0', '0:41:0:0', '0:42:0:0', '0:43:0:0', '0:44:0:0', '0:45:0:0', '0:46:0:0', '1:0:0:0', '1:1:0:0', '1:2:0:0', '1:3:0:0', '1:4:0:0', '1:5:0:0', '1:6:0:0', '1:7:0:0', '1:8:0:0', '1:9:0:0', '1:10:0:0', '1:11:0:0', '1:12:0:0', '1:13:0:0', '1:14:0:0', '1:15:0:0', '1:16:0:0', '1:17:0:0', '1:18:0:0', '1:19:0:0', '1:20:0:0', '1:21:0:0', '1:22:0:0', '1:23:0:0', '1:24:0:0', '1:25:0:0', '1:26:0:0', '1:27:0:0', '1:28:0:0', '1:29:0:0', '1:30:0:0', '1:31:0:0', '1:32:0:0', '1:33:0:0', '1:34:0:0', '1:35:0:0', '1:36:0:0', '

In [30]:
# datapoints

In [24]:
dimensions = data['structure']['dimensions']
series = data['dataSets'][0]['series']

In [26]:
index_cols = [x['name'] for x in dimensions['series']]

In [27]:
recs = []
name = data['structure']['name']

for i, v in series.items():
    idxs = list(map(int, i.split(':')))
    idx_dict = dict(zip(index_cols, idxs))
    for i, k in enumerate(idx_dict.keys()):
        assert i == dimensions['series'][i]['keyPosition']
        idx_dict[k] = dimensions['series'][i]['values'][idx_dict[k]]['id']
    for t, o in v['observations'].items():
        # TODO: might be the observations index is a list?
        year = int(dimensions['observation'][0]['values'][int(t)]['id'])

        rec = {}
        for k, v in idx_dict.items():
            rec[k] = v
        rec['year'] = year
        rec[name] = o[0]
        recs.append(rec)
    #break

In [28]:
rdf = pd.DataFrame.from_records(recs)

In [29]:
rdf.head()

,Aid (ODA) by sector and donor [DAC5],Aid type,Amount type,Donor,Sector,year
0,312.59,528,A,20005,100,1967
1,297.02,528,A,20005,100,1968
2,217.72,528,A,20005,100,1969
3,401.97,528,A,20005,100,1970
4,2248.84,528,A,20005,100,1971


In [31]:
rdf.columns = rdf.columns.map(to_concept_id)

In [32]:
rdf.head()

,aid_oda_by_sector_and_donor_dac5,aid_type,amount_type,donor,sector,year
0,312.59,528,A,20005,100,1967
1,297.02,528,A,20005,100,1968
2,217.72,528,A,20005,100,1969
3,401.97,528,A,20005,100,1970
4,2248.84,528,A,20005,100,1971


In [79]:
rdf.amount_type = rdf.amount_type.map(to_concept_id)

In [33]:
rdf.amount_type.unique()

array(['A', 'D'], dtype=object)

In [36]:
rdf[rdf.amount_type == 'D'].shape

(130905, 6)

In [37]:
rdf.aid_type.unique()

array(['528', '529', '530'], dtype=object)

In [38]:
# entities

In [46]:
dimensions['series'][3]

{'id': 'AMOUNT',
 'keyPosition': 3,
 'name': 'Amount type',
 'values': [{'id': 'A', 'name': 'Current Prices'},
  {'id': 'D', 'name': 'Constant Prices'}]}

In [55]:
entities_concepts = []
entities_dfs = {}

for d in dimensions['series']:
    name = d['name']
    concept = to_concept_id(name)
    
    value_df = pd.DataFrame.from_records(d['values'])
    value_df.columns = [concept, 'name']
    
    entities_concepts.append({'concept': concept, 'name': name, 'concept_type': 'entity_domain'})
    entities_dfs[concept] = value_df

In [56]:
pd.DataFrame.from_records(entities_concepts)

,concept,concept_type,name
0,donor,entity_domain,Donor
1,sector,entity_domain,Sector
2,aid_type,entity_domain,Aid type
3,amount_type,entity_domain,Amount type


In [77]:
entities_dfs['amount_type']

,amount_type,name
0,A,Current Prices
1,D,Constant Prices


In [69]:
concepts = [
    {'concept': 'oda', 'concept_type': 'measure', 'name': 'Aid (ODA) by sector and donor'},
    {'concept': 'year', 'concept_type': 'time', 'name': 'Year'},
    {'concept': 'name', 'concept_type': 'string', 'name': 'Name'}
]

In [73]:
cdf = pd.DataFrame.from_records([*concepts, *entities_concepts])

In [74]:
cdf

,concept,concept_type,name
0,oda,measure,Aid (ODA) by sector and donor
1,year,time,Year
2,name,string,Name
3,donor,entity_domain,Donor
4,sector,entity_domain,Sector
5,aid_type,entity_domain,Aid type
6,amount_type,entity_domain,Amount type


In [60]:
from pathlib import Path

In [78]:
for k, v in entities_dfs.items():
    path = Path('../', f'ddf--entities--{k}.csv')
    v[k] = v[k].map(to_concept_id)
    v.to_csv(path, index=False)

In [66]:
rdf = rdf.rename(columns={'aid_oda_by_sector_and_donor_dac5': 'oda'})

In [80]:
(rdf.set_index(['aid_type', 'amount_type', 'donor', 'sector', 'year'])
 .to_csv('../ddf--datapoints--oda--by--aid_type--amount_type--donor--sector--year.csv'))

In [75]:
cdf.to_csv('../ddf--concepts.csv', index=False)